In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from scipy.optimize import differential_evolution
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. โหลดข้อมูล
# ============================================================
df = pd.read_csv('experiment_design_27points.csv')
X = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']]
y = df.filter(like='Alpha_')

# เลือก 1-100 Hz
selected_hz = [f'Alpha_{i}Hz' for i in range(1, 101)]
y_selected = y[selected_hz]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================================
# 2. เปรียบเทียบโมเดลด้วย Cross-Validation
# ============================================================
print("=" * 60)
print("📊 เปรียบเทียบโมเดลด้วย Leave-One-Out Cross-Validation")
print("=" * 60)

# เลือก 5 ความถี่ทดสอบ
test_cols = ['Alpha_1Hz', 'Alpha_25Hz', 'Alpha_50Hz', 'Alpha_75Hz', 'Alpha_100Hz']

models = {
    'Polynomial (deg=2)': Pipeline([
        ('poly', PolynomialFeatures(degree=2)),
        ('ridge', Ridge(alpha=1.0))
    ]),
    'SVR (RBF)': Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(kernel='rbf', C=10, gamma='scale'))
    ]),
    'Random Forest': RandomForestRegressor(
        n_estimators=50, max_depth=5, random_state=42
    ),
    'XGBoost': XGBRegressor(
        n_estimators=30, max_depth=3, reg_lambda=1, verbosity=0, random_state=42
    ),
}

# ทดสอบแต่ละโมเดล
results = {}
for name, model in models.items():
    scores = []
    for col in test_cols:
        cv_score = cross_val_score(model, X, y_selected[col], cv=5, scoring='r2')
        scores.append(np.mean(cv_score))
    results[name] = np.mean(scores)
    print(f"  {name:25s}: R² = {results[name]:.4f}")

# หาโมเดลที่ดีที่สุด
best_model_name = max(results, key=results.get)
print(f"\n🏆 โมเดลที่ดีที่สุด: {best_model_name} (R² = {results[best_model_name]:.4f})")

# ============================================================
# 3. เทรนโมเดลที่ดีที่สุด
# ============================================================
print(f"\n🔄 เทรน {best_model_name} สำหรับทุกความถี่...")

# สร้างฟังก์ชันสร้างโมเดล
def create_model(name):
    if name == 'Polynomial (deg=2)':
        return Pipeline([
            ('poly', PolynomialFeatures(degree=2)),
            ('ridge', Ridge(alpha=1.0))
        ])
    elif name == 'SVR (RBF)':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('svr', SVR(kernel='rbf', C=10, gamma='scale'))
        ])
    elif name == 'Random Forest':
        return RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
    else:  # XGBoost
        return XGBRegressor(n_estimators=30, max_depth=3, reg_lambda=1, verbosity=0, random_state=42)

# เทรนโมเดลสำหรับแต่ละความถี่
trained_models = {}
for col in y_selected.columns:
    model = create_model(best_model_name)
    model.fit(X, y_selected[col])
    trained_models[col] = model

print(f"✅ เทรนเสร็จ {len(trained_models)} โมเดล")

# ============================================================
# 4. Inverse Optimization
# ============================================================
print("\n" + "=" * 60)
print("🔍 หาค่า Input ที่เหมาะสม...")
print("=" * 60)

# Target output
target_output = np.linspace(0.7, 0.98, len(selected_hz))

def predict_all(x):
    x_arr = np.array(x).reshape(1, -1)
    preds = [trained_models[col].predict(x_arr)[0] for col in y_selected.columns]
    return np.array(preds)

def objective(x):
    preds = predict_all(x)
    mse = np.mean((preds - target_output) ** 2)
    return mse

bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),
    (X['Proportion2'].min(), X['Proportion2'].max()),
    (X['Temp_C'].min(), X['Temp_C'].max()),
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()),
]

# Optimize
result = differential_evolution(
    objective, bounds,
    seed=42, maxiter=50, popsize=10,
    polish=True, disp=True
)

# ============================================================
# 5. ผลลัพธ์
# ============================================================
print("\n" + "=" * 50)
print("🏆 ค่า Input ที่เหมาะสม:")
print("=" * 50)
print(f"  Proportion1:   {result.x[0]:.4f}")
print(f"  Proportion2:   {result.x[1]:.4f}")
print(f"  Temp_C:        {result.x[2]:.4f}")
print(f"  Pressure_bar:  {result.x[3]:.4f}")
print("=" * 50)

final_pred = predict_all(result.x)
mse = np.mean((final_pred - target_output) ** 2)

print(f"\n📈 MSE: {mse:.6f}")
print(f"📈 RMSE: {np.sqrt(mse):.6f}")

# ตัวอย่างผล
print(f"\n📊 ตัวอย่างผลทำนาย:")
print(f"{'Hz':>6} | {'Target':>8} | {'Predicted':>10} | {'Error':>8}")
print("-" * 45)
for i in [0, 24, 49, 74, 99]:
    hz = i + 1
    err = abs(final_pred[i] - target_output[i])
    print(f"{hz:>6} | {target_output[i]:>8.4f} | {final_pred[i]:>10.4f} | {err:>8.4f}")

print("\n✅ เสร็จสิ้น!")
# ข้อมูล 27 จุด → ใช้ SVR หรือ Polynomial Regression
# ต้องการ Uncertainty → ใช้ GPR (แค่บางความถี่)
# ต้องการความเร็ว → ใช้ XGBoost หรือ Polynomial
# ต้องการความแม่นยำสูงสุด → ลองรันโค้ดด้านบนเพื่อเปรียบเทียบ